In [1]:
from pyscf.lib import krylov
from pyscf import gto, scf
import scipy
import inspect
import pyscf.qmmm
import pyscf.dft
import numpy as np
angstrom = 1 / 0.52917721067
from matplotlib import pyplot as plt
from functools import reduce
from numpy.linalg import inv 
from pyscf.scf import cphf 

def fc(calc,deltaZ):
    mf = pyscf.qmmm.mm_charge(calc, calc.mol.atom_coords(), deltaZ)  # now is add_mm_charge
    class NoSelfQMMM(mf.__class__):
        def energy_nuc(self):
            q = self.mol.atom_charges().astype(np.float).copy()
            q1 =q+ np.asarray(deltaZ) 
            return self.mol.energy_nuc(q1)
    return(NoSelfQMMM(mf,mf.mm_mol))

def DeltaV(mol,dL):
    mol.set_rinv_orig_(mol.atom_coords()[0])
    dV=mol.intor('int1e_rinv')*dL[0]
    mol.set_rinv_orig_(mol.atom_coords()[1])
    dV+=mol.intor('int1e_rinv')*dL[1]
    return -dV

In [30]:
mol=gto.Mole(atom="H 0 0 0 ;H 0 0 1.2", unit="Bohr", basis= "sto-3G")
mf=scf.RHF(mol)
mf.scf()
dL=.01
mf1=fc(mf,[dL,-dL])
mf1.scf()
mf2=fc(mf,[-dL,dL])
mf2.scf()
dV=DeltaV(mol,[dL,-dL])
h2=mf.get_veff()
dh2=(mf1.get_veff()-mf2.get_veff())/2
J,K=mf.get_jk()
g_ijkl=mf.mol.intor('int2e_sph')
g_h2=g_ijkl-np.swapaxes(g_ijkl,1,2)/2
nocc= mol.nelec[0]


C=mf.mo_coeff
nmo=C.shape[0]
nvir=nmo-nocc
S=mf.get_ovlp()
e=mf.mo_energy
E=np.diag(e)
O=np.diag(mf.mo_occ)
C1=mf1.mo_coeff
P=mf.make_rdm1()
P1=mf1.make_rdm1()
P2=mf2.make_rdm1() 
dP=(P1-P2)/2
dP2=(P1-2*P+P2)/dL**2
dC=(abs(C1)*C/abs(C)-C)
dV_mo=C.T@dV@C

Initialize <pyscf.gto.mole.Mole object at 0x7fa310486750> in <pyscf.scf.hf.RHF object at 0x7fa3104869d0>


converged SCF energy = -1.11033388268018
converged SCF energy = -1.11050145845196
converged SCF energy = -1.11050145845196


In [31]:
def test(u_): # check density matrices comparing to the one ontained by FD
    ut=np.zeros((nmo,nmo))
    ut[nocc:,:nocc]=u_.reshape(nvir,nocc)
    ut[:nocc,nocc:]=-u_.reshape(nvir,nocc).T
    dP_app=C@(ut@O-O@ut)@C.T    
    print(np.linalg.norm(dP))
    print(np.linalg.norm(dP_app))
    print(np.linalg.norm(dP_app-dP))

In [32]:
em1,em2=np.meshgrid(e,e)
emeshvo=(em1-em2)[nocc:,:nocc]
emeshf=emeshvo.flatten()
G_cphf=g_ijkl.copy()
G_cphf=4*G_cphf -G_cphf.swapaxes(3,1)-G_cphf.swapaxes(1,2)
G_cphf=G_cphf@C
G_cphf=(G_cphf.swapaxes(2,3)@C).swapaxes(3,2)
G_cphf=(G_cphf.swapaxes(1,3)@C).swapaxes(3,1)
G_cphf=(G_cphf.swapaxes(0,3)@C).swapaxes(3,0)

G_cphfvovo=G_cphf.copy()[nocc:,:nocc,nocc:,:nocc]    #aibj

In [33]:
dVvo=(dV_mo[nocc:,:nocc])
dVf=dVvo.flatten()

In [34]:
def emul(_u):
    return _u*emeshf -_u #from U gives dV

In [35]:
def gmul(_u):
    return np.einsum('ijkl,kl->ij',G_cphfvovo,_u.reshape(nvir,nocc)).flatten()

In [36]:
def amul(_u):
    _u=_u.flatten()
    return emul(_u)-gmul(_u)

In [37]:
uk=krylov(amul,dVf, max_cycle=100)

In [38]:
np.max(amul(uk)+uk-dVf)

0.0

In [39]:
test(uk)

0.023871933740730066
0.023873114928908826
1.1811881787601493e-06


In [40]:
uk

array([0.00578073])

In [41]:
u_cphf=cphf.solve_nos1(gmul, e, mf.mo_occ, dVvo)[0]
test(u_cphf)

0.023871933740730066
0.023873114928908826
1.1811881787601493e-06


In [42]:
u_cphf.shape

(1, 1)

In [43]:
vresp=mf.gen_response(hermi=1)
np.allclose(vresp(P),h2)

True

In [44]:
from pyscf import lib
def gen_vind( mf, mo_coeff, mo_occ):  #polarizability
        '''Induced potential'''
        vresp = mf.gen_response(hermi=1)
        occidx = mo_occ > 0
        orbo = mo_coeff[:, occidx]
        nocc = orbo.shape[1]
        nao, nmo = mo_coeff.shape
        def vind(mo1):
            dm1 = lib.einsum('xai,pa,qi->xpq', mo1.reshape(-1,nmo,nocc), mo_coeff,orbo.conj())
            dm1 = (dm1 + dm1.transpose(0,2,1).conj()) * 2
            v1mo = lib.einsum('xpq,pi,qj->xij', vresp(dm1), mo_coeff.conj(), orbo)
            return v1mo.ravel()
        return vind

In [45]:
vind=gen_vind(mf, mf.mo_coeff,mf.mo_occ)

In [46]:
vind(dV[:,:nocc])

array([-0.00693429,  0.        ])

In [47]:
"""   CPHF.nos1 -code 

e_a = mo_energy[mo_occ==0]
e_i = mo_energy[mo_occ>0]
e_ai = 1 / lib.direct_sum('a-i->ai', e_a, e_i)
mo1base = h1 * -e_ai

def vind_vo(mo1):  #mo1 =dC 
    v = fvind(mo1.reshape(h1.shape)).reshape(h1.shape)
    v *= e_ai
    return v.ravel()
mo1 = lib.krylov(vind_vo, mo1base.ravel(),
                 tol=tol, max_cycle=max_cycle, hermi=hermi, verbose=log)
log.timer('krylov solver in CPHF', *t0)
return mo1.reshape(h1.shape), None
"""

"   CPHF.nos1 -code \n\ne_a = mo_energy[mo_occ==0]\ne_i = mo_energy[mo_occ>0]\ne_ai = 1 / lib.direct_sum('a-i->ai', e_a, e_i)\nmo1base = h1 * -e_ai\n\ndef vind_vo(mo1):  #mo1 =dC \n    v = fvind(mo1.reshape(h1.shape)).reshape(h1.shape)\n    v *= e_ai\n    return v.ravel()\nmo1 = lib.krylov(vind_vo, mo1base.ravel(),\n                 tol=tol, max_cycle=max_cycle, hermi=hermi, verbose=log)\nlog.timer('krylov solver in CPHF', *t0)\nreturn mo1.reshape(h1.shape), None\n"